In [1]:
import glob,cv2,os,re
from mylab.developing.single_cell_analysis.Mfunctions import *

In [3]:
raw_data_dir = r"\\10.10.46.135\Lab_Members\_Lab Data Analysis\02_Linear_Track\Miniscope_Linear_Track\batch3\Results_*\part*"
sessions = glob.glob(os.path.join(raw_data_dir,"session*.pkl"))
sessions

# for session in sessions:
#     divide_sessions_into_trials(session)

['\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session1.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session2.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session3.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session4.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session5.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-finish\\part1\\session6.pkl',
 '\\\\10.10.46.135\\Lab_Members\\_Lab Data Analysis\\02_Linear_Track\\Miniscope_Linear_Track\\batch3\\Results_201033-f

In [2]:
trials = construct_trial_lists(mouse_id="201033",part="1",day="20200722")

session = concatenate_trials(trials)

201033_part1_index20200722-144326_session4_trial1.pkl is loaded
201033_part1_index20200722-144326_session4_trial2.pkl is loaded
201033_part1_index20200722-144326_session4_trial3.pkl is loaded
201033_part1_index20200722-144326_session4_trial4.pkl is loaded
201033_part1_index20200722-144326_session4_trial5.pkl is loaded
201033_part1_index20200722-144326_session4_trial6.pkl is loaded
201033_part1_index20200722-144326_session4_trial7.pkl is loaded
201033_part1_index20200722-144326_session4_trial8.pkl is loaded
201033_part1_index20200722-144326_session4_trial9.pkl is loaded
201033_part1_index20200722-144326_session4_trial10.pkl is loaded
201033_part1_index20200722-144326_session4_trial11.pkl is loaded
201033_part1_index20200722-144326_session4_trial12.pkl is loaded
201033_part1_index20200722-145320_session5_trial1.pkl is loaded
201033_part1_index20200722-145320_session5_trial2.pkl is loaded
201033_part1_index20200722-145320_session5_trial3.pkl is loaded
201033_part1_index20200722-145320_ses

In [6]:
from mylab.ana.miniscope.context_exposure.Cminiana import AnaMini as AM
from mylab.ana.miniscope.context_exposure.ana_funtions import *

In [4]:
s = AM(session)

In [8]:
def cellids_Context(s,*args,**kwargs):
    """
    s.add_Context()
    s.add_Trial_Num_Process()
    s.add_alltrack_placebin_num(placebins=[4,4,30,4,4,4])
    """
    print("FUNC::cellids_Context")
    df,index = s.trim_df(*args,**kwargs)
    df = df[index]


    Trial_Num = s.result["Trial_Num"][index]
    Context= s.result["Context"][index]

    meanfr_df = df.groupby([Trial_Num,Context]).mean()
    ctx_meanfr = meanfr_df.groupby(["Context"]).mean()

    result = {}
    result["meanfr_df"] = meanfr_df
    result["ctx_meanfr"] = ctx_meanfr

    if len(set(Context)) > 1:
        for ctxes in itertools.combinations(set(Context),2):
            a,b = ctxes
            idx_a = meanfr_df.index.get_level_values(level="Context")==a
            idx_b = meanfr_df.index.get_level_values(level="Context")==b
            ctx_pvalue = meanfr_df.apply(func=lambda x: stats.ranksums(x[idx_a],x[idx_b])[1],axis=0)
            CSI = (ctx_meanfr.loc[a,:]-ctx_meanfr.loc[b,:])/(ctx_meanfr.loc[a,:]+ctx_meanfr.loc[b,:])

            ContextA_cells=[]
            ContextB_cells=[]
            non_context_cells=[]

            for cellid,csi, p in zip(ctx_meanfr.columns,CSI,ctx_pvalue):
                if p>=0.05:
                    non_context_cells.append(cellid)
                else:
                    if csi>0:
                        ContextA_cells.append(cellid)
                    elif csi<0:
                        ContextB_cells.append(cellid)
                    else:
                        print("meanfr of cell %s is equal in Context %s and Context %s"%(cellid,a,b))
                        non_context_cells.append(cellid)

            result["ctx%s_%s"%(a,b)]={
            "ctx_pvalue":ctx_pvalue,
            "CSI":CSI,
            "context%s_cells"%a:ContextA_cells,
            "context%s_cells"%b:ContextB_cells,
            "non_context_cells":non_context_cells
            }


    return result

In [9]:
cellids_Context(s,"S_dff",placebin=np.arange(8,38))

FUNC::cellids_Context
trim_index was initialed by Trial_Num>=0
trim_index : placebins are limited in [ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
 32 33 34 35 36 37]
'S_dff' is taken as original self.df
trim_df : df was trimmed.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [18]:
s.result["aligned_behave2ms"]["Trial_Num"]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [15]:
s.result["behavelog_info"]

    Trial_Num  Enter_ctx  Exit_ctx  Choice_class  Left_choice  Right_choice
0           1          0         0             1            1             0
1           2          1         1             0            2             0
2           3          0         0             1            3             0
3           4          0         0             1            4             0
4           5          1         1             0            5             0
..        ...        ...       ...           ...          ...           ...
45         46          1         1             0           46             0
46         47          0         0             1           47             0
47         48          1         1             0           48             0
48         49          0         0             1           49             0
49         50          1         1             0           50             0

[62 rows x 6 columns]
